In [3]:
import tensorflow as tf 
from tensorflow import keras
import keras_tuner as kt

In [4]:
(img_train, label_train),(img_test, label_train)= keras.datasets.fashion_mnist.load_data()

In [5]:
img_train.shape

(60000, 28, 28)

In [6]:
img_test.shape

(10000, 28, 28)

In [7]:
import numpy as np 
np.max(img_train)

255

In [8]:
np.min(img_train)

0

In [9]:
a=np.max(img_test)
print(a)
b=np.min(img_test)
print(b)

255
0


In [10]:
img_train=img_train.astype('float32')/255.0
img_test=img_test.astype('float32')/255.0

In [11]:
a=np.max(img_test)
print(a)
b=np.min(img_test)
print(b)

1.0
0.0


In [12]:
a=np.max(img_train)
print(a)
b=np.min(img_train)
print(b)

1.0
0.0


In [13]:
def model_builder(hp):
    model=keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    hp_units=hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))
    hp_learning_rate=hp.Choice('learnig_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model
    


In [14]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='tu')

INFO:tensorflow:Reloading Tuner from my_dir/tu/tuner0.json


In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [16]:
tuner.search(img_train, label_train, epochs=5, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
352               |?                 |units
0.001             |?                 |learnig_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round



2023-02-21 16:58:15.022767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-21 16:58:15.022973: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-21 16:58:15.022989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IGN-BLR-LP-0412): /proc/driver/nvidia/version does not exist
2023-02-21 16:58:15.023558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/home

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/home/pavanb/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 250, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/pavanb/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 215, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/pavanb/.local/lib/python3.8/site-packages/keras_tuner/tuners/hyperband.py", line 404, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/pavanb/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 286, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/pavanb/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 213, in _build_and_fit_model
    results = self.hypermodel.fit(hp, 

In [17]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

NameError: name 'best_hps' is not defined

In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

NameError: name 'best_hps' is not defined

In [19]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

NameError: name 'hypermodel' is not defined